In [10]:
import s2sphere as s2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import geometry
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

COLUMN_CELL_LEVEL = "Cell Level"
COLUMN_CELL_TOKEN = "Cell Token"
COLUMN_LAT = "Lat"
COLUMN_LONG = "Long"
COLUMN_POSTAL_CODE = "PostalCode"

# ==========================================UTILITY============================================
def plotLatLong(row):
    latlng = s2.LatLng.from_degrees(float(row['Lat']), float(row['Long']))
    cell = s2.CellId.from_lat_lng(latlng)
    row[COLUMN_CELL_LEVEL] = cell.level()
    row[COLUMN_CELL_TOKEN] = cell.to_token()
    return row

def formatData(df):
    # Change column names
    df.columns = ['LatLong', COLUMN_POSTAL_CODE]
    
    # Split LatLong into different columns
    df[COLUMN_LAT], df[COLUMN_LONG] = df['LatLong'].str.split(',', 1).str
    df = df[df.columns[1:]]
    df[COLUMN_CELL_LEVEL] = None
    df[COLUMN_CELL_TOKEN] = None
#     df = df.head(n=10)
#     echo data
#     print df.head(n=6)
    return df
# ==========================================UTILITY============================================


df = pd.read_csv("~/Downloads/pincodes_with_lat_long.csv")
df = formatData(df)

print("Total points : %s" %df.shape[0])


# Comparing with rect-region with 2 points
# row1 = df.iloc[0]
# row2 = df.iloc[1]
# LL1 = s2.LatLng.from_degrees(float(row1['Lat']), float(row1['Long']))
# LL2 = s2.LatLng.from_degrees(float(row2['Lat']), float(row2['Long']))
# region_rect = s2.LatLngRect.from_point_pair(LL1, LL2)
# print("Rect area : %s" % region_rect.area())

# Create a cap with all lat-longs
cap = s2.Cap.empty()
for index, row in df.iterrows():
    point = s2.LatLng.from_degrees(float(row['Lat']), float(row['Long'])).to_point()
    
    cap.add_point(point)
#     print("Cap rect area : %s" % cap.get_rect_bound().area()) 
    
# Get covering of the cap
coverer = s2.RegionCoverer()
coverer.min_level = 10
coverer.max_level = 15
coverer.max_cells= 500
covering = coverer.get_covering(cap.get_rect_bound())

print("Total cells in the region : %s" %len(covering))


# geoms = []
# for cellid in covering:
#     new_cell = s2.Cell(cellid)
#     vertices = []
#     for i in xrange(0, 4):
#         vertex = new_cell.get_vertex(i)
#         latlng = s2.LatLng.from_point(vertex)
#         vertices.append((latlng.lat().degrees,
#                          latlng.lng().degrees))
#     geo = Polygon(vertices)
#     geoms.append(geo)
# print "Total Geometries: {}".format(len(geoms))
    

    



Total points : 24260
Total cells in the region : 494
